In [2]:
from datasets import load_dataset

dataset = load_dataset('snli')

labels = dataset.unique("label").keys()
num_labels = len(labels)
# label -1 is used if the gold label is missing, see: https://github.com/huggingface/datasets/issues/296
# therefore we remove all entries with label -1
dataset = dataset.filter(lambda x: x["label"] != -1)

# shuffle the dataset to avoid any bias
dataset = dataset.shuffle(seed=42)

# rename label to labels, because the model expects a column named labels
dataset = dataset.rename_column("label", "labels")

print(f"{labels=}")
print(f"{num_labels=}")

labels=dict_keys(['test', 'validation', 'train'])
num_labels=3


In [3]:
from transformers import AutoModelForSequenceClassification

import torch.nn as nn

class NLIClassifier(nn.Module):
    def __init__(self, model, num_labels=3, freeze_bert=False):
        super(NLIClassifier, self).__init__()
        self.bert = model
        # freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

        for param in self.bert.classifier.parameters():
            param.requires_grad = True

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # Extract [CLS] token representation
        logits = self.classifier(cls_output)
        return logits

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-cased",
    num_labels=num_labels,
    torch_dtype="auto",
)
nli_model = NLIClassifier(model, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

# tokenize dataset
tokenized_dataset = dataset.map(lambda x: tokenizer(x["premise"], x["hypothesis"], add_special_tokens=True, truncation=True), batched=True)

Map:   0%|          | 0/9824 [00:00<?, ? examples/s]

Map:   0%|          | 0/9842 [00:00<?, ? examples/s]

Map:   0%|          | 0/549367 [00:00<?, ? examples/s]

In [5]:
import torch

criterion = nn.CrossEntropyLoss()

In [6]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# remove all text columns, because they are already tokenized
tokenized_dataset = tokenized_dataset.remove_columns(["premise", "hypothesis"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
traindataloader = DataLoader(tokenized_dataset["train"], batch_size=32, shuffle=True, collate_fn=data_collator)

In [14]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [15]:
from torch.optim import AdamW
# optimizer = AdamW(nli_model.parameters(),lr=5e-5)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, nli_model.parameters()), lr=5e-5)

from transformers import get_scheduler
num_epochs=3
num_training_steps = num_epochs * len(traindataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
nli_model.to(device)
nli_model.train()

for epoch in range(num_epochs):
    for batch in traindataloader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = criterion(outputs.logits, batch["labels"])
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        progress_bar.update(1)

  0%|          | 0/51504 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
testdataloader = DataLoader(tokenized_dataset["test"].select(range(10)), batch_size=32, shuffle=True, collate_fn=data_collator)

In [18]:
import evaluate

metric = evaluate.load("accuracy")
nli_model.eval()
for batch in testdataloader:
    # for model inference we don't need gradients
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.5}